<a href="https://colab.research.google.com/github/delhiiitian/CrossSell/blob/main/nn_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.preprocessing import  LabelEncoder,OrdinalEncoder,MinMaxScaler
from keras.models import Sequential
import keras
from tensorflow.keras.utils import to_categorical 
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, accuracy_score
# from bayes_opt import BayesianOptimization
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
traindf = pd.read_csv('train_full_new.csv',index_col=0)
testdf = pd.read_csv('test_full_new.csv',index_col=0)

testdf.drop(columns=['season'],inplace=True)

In [3]:
# one hot
# train = pd.get_dummies(traindf,columns=['edible-poisonous', 'cap-shape', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
#        'stem-color', 'has-ring', 'ring-type', 'habitat'],drop_first=True)
# test = pd.get_dummies(testdf,columns=['edible-poisonous', 'cap-shape', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
#        'stem-color', 'has-ring', 'ring-type', 'habitat',],drop_first=True)


train1, test1 = train_test_split(traindf, test_size=0.2,stratify=traindf['season'])

# train, val = train_test_split(train, test_size=0.2)

X_train,y_train = train1.loc[:, train1.columns != 'season'],train1['season']
X_test,y_test = test1.loc[:, test1.columns != 'season'],test1['season']

categorical_columns = ['edible-poisonous', 'cap-shape', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-color',
       'stem-color', 'has-ring', 'ring-type', 'habitat']

numerical_columns = ['cap-diameter','stem-height','stem-width']

column_trans = make_column_transformer((OrdinalEncoder(),categorical_columns),(MinMaxScaler(),numerical_columns))

column_trans.fit(X_train)
X_train = column_trans.transform(X_train)
X_test = column_trans.transform(X_test)

# # LE
# # prepare input data
# def prepare_inputs(X_train, X_test):
# 		oe = OrdinalEncoder()
# 		oe.fit(X_train)
# 		X_train_enc = oe.transform(X_train,)
# 		X_test_enc = oe.transform(X_test)
# 		return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
		le = LabelEncoder()
		le.fit(y_train)
		y_train_enc = le.transform(y_train)
		y_test_enc = le.transform(y_test)
		return y_train_enc, y_test_enc

# # prepare input data
# # X_train_enc, X_test_enc = prepare_inputs(tr1, ts1)
# # prepare output data
y_train, y_test = prepare_targets(y_train, y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [4]:
X_train.shape[1]

13

In [34]:
model = Sequential()
model.add(Dense(13, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.01))
# model.add(Dense(128, activation='tanh'))
# model.add(Dropout(0.01))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.01))
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, epochs=20)
model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.2546 - accuracy: 0.4568
Epoch 2/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0588 - accuracy: 0.4802
Epoch 3/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0349 - accuracy: 0.4890
Epoch 4/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0243 - accuracy: 0.4891
Epoch 5/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0177 - accuracy: 0.4923
Epoch 6/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0092 - accuracy: 0.4948
Epoch 7/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0060 - accuracy: 0.4934
Epoch 8/20
1069/1069 [==============================] - 2s 2ms/step - loss: 1.0009 - accuracy: 0.4950
Epoch 9/20
1069/1069 [==============================] - 2s 2ms/step - loss: 0.9981 - accuracy: 0.4973
Epoch 10/20
1069/1069 [==============================] - 2s 2ms/step - loss: 0.995

[0.9859254956245422, 0.48374268412590027]

In [36]:
model.get_weights()

[array([[-1.70851469e-01, -1.03469968e+00, -3.80963117e-01,
         -2.54231721e-01, -2.06317306e-01,  3.49227846e-01,
         -5.74401803e-02,  2.93359518e-01,  7.03506291e-01,
         -7.35124707e-01,  1.65661111e-01, -5.46644688e-01,
          6.54761732e-01],
        [-1.03611916e-01, -6.03132248e-01, -6.65289263e-05,
         -3.24966341e-01, -1.03642397e-01, -4.09573130e-02,
         -1.70841575e-01,  2.83463802e-02,  1.79197565e-01,
         -3.67443025e-01, -1.77004859e-01,  4.71140333e-02,
          2.42285803e-01],
        [-3.20174366e-01, -5.95638230e-02,  5.60716726e-02,
         -4.15114611e-01,  1.77546423e-02, -1.64731890e-01,
          2.91610658e-01, -1.32176414e-01,  3.18237324e-03,
         -6.90885186e-02,  3.79900374e-02,  3.38792950e-02,
         -1.27423648e-02],
        [-4.47394490e-01,  2.02130601e-01, -1.21159554e+00,
          2.96657711e-01,  8.00313354e-02,  7.93244660e-01,
          6.01003110e-01, -1.13698411e+00,  8.96567702e-01,
          1.4292113

In [37]:
traindf

,edible-poisonous,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,p,5.75,x,n,t,s,w,6.13,14.93,w,f,f,d,a
1,p,2.27,o,g,f,f,f,0.00,0.00,f,f,f,d,s
2,p,5.59,c,y,t,x,p,5.47,11.27,w,f,f,d,a
3,p,0.74,x,p,f,a,n,4.33,0.98,k,f,f,d,a
4,p,9.60,x,n,f,a,w,10.84,28.52,w,f,f,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42743,p,2.66,b,n,f,a,n,8.08,2.35,g,f,f,g,a
42744,p,9.18,x,n,f,x,w,9.83,11.96,w,f,f,d,u
42745,e,3.86,b,n,f,a,g,6.51,4.45,g,f,f,d,w
42746,p,3.39,b,n,f,e,k,10.32,5.47,w,t,z,g,u
